# Iphone Resell-value Analysis 
## Importing all dependencies

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

## Importing scraped data from ebay-terapeak

In [2]:
raw_data = pd.read_csv('MyScrape.csv')

## Setting up the derived columns and doing minor data preparation

In [3]:
raw_data['Model'] = raw_data['web-scraper-start-url'].str.extract(r'aspect=Modell%3A%3A%3A([^&]+)')
raw_data['Storage'] = raw_data['web-scraper-start-url'].str.extract(r'aspect=Speicherkapazit%C3%A4t%3A%3A%3A([^&]+)')
raw_data['Model'] = raw_data['Model'].str.replace('+', ' ')
raw_data['Storage'] = raw_data['Storage'].str.replace('+', ' ')
raw_data['Condition'] = 'gebraucht'

conditions = {
    'neu': 'neu',
    'neu sonstige': 'neu sonstige',
    'zertifiziert refurbished': 'zertifiziert refurbished',
    'hervorragend refurbished': 'hervorragend refurbished',
    'sehr gut refurbished': 'sehr gut refurbished',
    'gut refurbished': 'gut refurbished',
    'gebraucht': 'gebraucht',
    'defekt': 'defekt'
}
for condition, keyword in conditions.items():
    raw_data.loc[raw_data['Listing'].str.contains(keyword, case=False), 'Condition'] = condition

raw_data.drop(columns=['web-scraper-order', 'web-scraper-start-url', 'Listing'], inplace=True)

common_versions = {
    'Apple iPhone 11 Pro Max': '20.09.2019',
    'Apple iPhone 11 Pro': '20.09.2019',
    'Apple iPhone XS': '26.10.2018',
    'Apple iPhone XR': '26.10.2018',
    'Apple iPhone 11': '20.09.2019',
    'Apple iPhone X': '03.11.2017',
    'Apple iPhone 12': '23.10.2020',
    'Apple iPhone 12 Pro': '23.10.2020',
    'Apple iPhone 12 Pro Max': '23.10.2020',
    'Apple iPhone 12 mini': '23.10.2020',
    'Apple iPhone 13 mini': '24.09.2021',
    'Apple iPhone 13 Pro': '24.09.2021',
    'Apple iPhone 13 Pro Max': '24.09.2021',
    'Apple iPhone 13': '24.09.2021',
    'Apple iPhone 14 Pro': '16.09.2022',
    'Apple iPhone 14 Pro Max': '16.09.2022',
    'Apple iPhone 14': '16.09.2022'
}
raw_data['Release_date'] = raw_data['Model'].map(common_versions)

release_prices = {
    'Apple iPhone 11 Pro Max 64 GB': 1249,
    'Apple iPhone 11 Pro Max 256 GB': 1419,
    'Apple iPhone 11 Pro Max 512 GB': 1649,

    'Apple iPhone 11 Pro 64 GB': 1149,
    'Apple iPhone 11 Pro 256 GB': 1319,
    'Apple iPhone 11 Pro 512 GB': 1549,

    'Apple iPhone XS 64 GB': 999,
    'Apple iPhone XS 256 GB': 1149,
    'Apple iPhone XS 512 GB': 1349,

    'Apple iPhone XR 64 GB': 749,
    'Apple iPhone XR 128 GB': 799,
    'Apple iPhone XR 256 GB': 899,

    'Apple iPhone 11 64 GB': 699,
    'Apple iPhone 11 128 GB': 749,
    'Apple iPhone 11 256 GB': 849,

    'Apple iPhone X 64 GB': 999,
    'Apple iPhone X 256 GB': 1149,

    'Apple iPhone 12 64 GB': 829,
    'Apple iPhone 12 128 GB': 879,
    'Apple iPhone 12 256 GB': 979,

    'Apple iPhone 12 Pro 128 GB': 999,
    'Apple iPhone 12 Pro 256 GB': 1099,
    'Apple iPhone 12 Pro 512 GB': 1299,

    'Apple iPhone 12 Pro Max 128 GB': 1099,
    'Apple iPhone 12 Pro Max 256 GB': 1199,
    'Apple iPhone 12 Pro Max 512 GB': 1399,

    'Apple iPhone 12 mini 64 GB': 699,
    'Apple iPhone 12 mini 128 GB': 749,
    'Apple iPhone 12 mini 256 GB': 849,

    'Apple iPhone 13 mini 128 GB': 749,
    'Apple iPhone 13 mini 256 GB': 869,
    'Apple iPhone 13 mini 512 GB': 1099,

    'Apple iPhone 13 128 GB': 849,
    'Apple iPhone 13 256 GB': 969,
    'Apple iPhone 13 512 GB': 1199,

    'Apple iPhone 13 Pro 128 GB': 999,
    'Apple iPhone 13 Pro 256 GB': 1099,
    'Apple iPhone 13 Pro 512 GB': 1299,
    'Apple iPhone 13 Pro 1 TB': 1499,

    'Apple iPhone 13 Pro Max 128 GB': 1099,
    'Apple iPhone 13 Pro Max 256 GB': 1199,
    'Apple iPhone 13 Pro Max 512 GB': 1399,
    'Apple iPhone 13 Pro Max 1 TB': 1599,

    'Apple iPhone 14 128 GB': 849,
    'Apple iPhone 14 256 GB': 969,
    'Apple iPhone 14 512 GB': 1199,

    'Apple iPhone 14 Pro 128 GB': 999,
    'Apple iPhone 14 Pro 256 GB': 1099,
    'Apple iPhone 14 Pro 512 GB': 1299,
    'Apple iPhone 14 Pro 1 TB': 1499,

    'Apple iPhone 14 Pro Max 128 GB': 1099,
    'Apple iPhone 14 Pro Max 256 GB': 1199,
    'Apple iPhone 14 Pro Max 512 GB': 1399,
    'Apple iPhone 14 Pro Max 1 TB': 1599
}

raw_data['Combined'] = raw_data['Model'] + " " + raw_data['Storage']
raw_data['release_price'] = raw_data['Combined'].map(release_prices)
raw_data = raw_data.drop(columns=['Combined'])

raw_data['Date'] = pd.to_datetime(raw_data['Date'], format='%m/%d/%Y')
raw_data['Release_date'] = pd.to_datetime(raw_data['Release_date'], format='%d.%m.%Y')
raw_data['Time_used_days'] = (raw_data['Date'] - raw_data['Release_date']).dt.days

## Data exploration and minor analysis

In [6]:
working_data = raw_data

,Price,Sale_Format,Shipping_Cost,Shipping_percent,Shipping,Total sold,Item_Revenue,Bids,Date,Model,Storage,Condition,Release_date,release_price,Time_used_days
0,912.64,Fixed price,0.0,1.0,Free shipping,1,912.64,–,2020-11-23,Apple iPhone 11 Pro Max,64 GB,gebraucht,2019-09-20,1249,430
1,1231.63,Fixed price,0.0,1.0,Free shipping,1,1231.63,–,2020-11-23,Apple iPhone 11 Pro Max,256 GB,gebraucht,2019-09-20,1419,430
2,1231.63,Fixed price,0.0,1.0,Free shipping,1,1231.63,–,2020-11-23,Apple iPhone 11 Pro Max,512 GB,gebraucht,2019-09-20,1649,430
3,970.84,Fixed price,0.0,1.0,Free shipping,2,1941.68,–,2020-11-23,Apple iPhone 11 Pro,64 GB,gebraucht,2019-09-20,1149,430
4,1095.31,Fixed price,0.0,1.0,Free shipping,4,4381.24,–,2020-11-23,Apple iPhone 11 Pro,512 GB,gebraucht,2019-09-20,1549,430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81267,1282.98,Fixed price,0.0,1.0,Free shipping,9,11546.84,–,2023-11-23,Apple iPhone 14 Pro Max,256 GB,neu,2022-09-16,1199,433
81268,977.93,Fixed price,0.0,1.0,Free shipping,1,977.93,–,2023-11-23,Apple iPhone 14 Pro Max,256 GB,gebraucht,2022-09-16,1199,433
81269,977.88,Fixed price,0.0,1.0,Free shipping,1,977.88,–,2023-11-23,Apple iPhone 14 Pro Max,256 GB,gebraucht,2022-09-16,1199,433
81270,1260.76,Fixed price,0.0,1.0,Free shipping,1,1260.76,–,2023-11-23,Apple iPhone 14 Pro Max,256 GB,neu,2022-09-16,1199,433
